In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline

np.random.seed(1)

In [26]:
def initialize_parameters_deep(layer_dims):
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        
        parameters[f"W{l}"] = np.random.randn(layer_dims[l], layer_dims[l-1]) * .01
        parameters[f"b{l}"] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [3]:
def linear_forward(A, W, b):
    
    Z = W@A + b
    
    cache = (A, W, b)
    
    return Z, cache

In [6]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    elif activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [8]:
def L_model_forward(X, parameters):
    
    L = len(parameters) // 2
    caches = []
    A = X
    
    for l in range(1, L):

        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters[f"W{l}"], parameters[f"b{l}"], "relu")
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters[f"W{L}"], parameters[f"b{L}"], "sigmoid")
    caches.append(cache)
    
    return AL, caches

In [28]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    
    cost = -1/m * (Y@np.log(AL).T + (1-Y)@np.log(1-AL).T)
    
    cost = np.squeeze(cost)
    
    return cost

In [30]:
def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    
    m = A_prev.shape[1]
    
    dW = 1/m * (dZ@A_prev.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = W.T@dZ
    
    return dA_prev, dW, db

In [31]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [32]:
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = -(Y/AL) + ((1-Y)/(1-AL))
    current_cache = caches[L-1]
    
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, "sigmoid")
    
    grads[f"dA{L-1}"] = dA_prev_temp
    grads[f"dW{L}"] = dW_temp
    grads[f"db{L}"] = db_temp
    
    for l in reversed(range(L-1)):
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dA_prev_temp, current_cache, "relu")
        
        grads[f"dA{l}"] = dA_prev_temp
        grads[f"dW{l+1}"] = dW_temp
        grads[f"db{l+1}"] = db_temp
        
    return grads

In [33]:
def update_parameters(params, grads, learning_rate):
    
    parameters = params.copy()
    L = len(parameters) // 2
    
    for l in range(L):
        parameters[f"W{l+1}"] = parameters[f"W{l+1}"] - learning_rate * grads[f"dW{l+1}"]
        parameters[f"b{l+1}"] = parameters[f"b{l+1}"] - learning_rate * grads[f"db{l+1}"]
        
    return parameters

In [46]:
#Custom function. Not available in the original/corresponding notebook.
def predict(X, Y, parameters):
    
    AL, _ = L_model_forward(X, parameters)
    predictions = AL > .5
    accuracy = np.average(predictions == Y)
    print(f"Accuracy: {accuracy}")
    
    return accuracy

In [15]:
train_x = np.load("../../catvnoncat/data/processed/train_set_x.npy")
train_y = np.load("../../catvnoncat/data/processed/train_set_y.npy")
test_x = np.load("../../catvnoncat/data/processed/test_set_x.npy")
test_y = np.load("../../catvnoncat/data/processed/test_set_y.npy")

In [21]:
layers_dims = [12288, 20, 7, 5, 1]

In [22]:
# GRADED FUNCTION: L_layer_model (copied from the corresponding notebook)

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.
    #(≈ 1 line of code)
    # parameters = ...
    # YOUR CODE STARTS HERE
    parameters = initialize_parameters_deep(layers_dims)
    
    # YOUR CODE ENDS HERE
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        #(≈ 1 line of code)
        # AL, caches = ...
        # YOUR CODE STARTS HERE
        AL, caches = L_model_forward(X, parameters)
        
        # YOUR CODE ENDS HERE
        
        # Compute cost.
        #(≈ 1 line of code)
        # cost = ...
        # YOUR CODE STARTS HERE
        cost = compute_cost(AL, Y)
        
        # YOUR CODE ENDS HERE
    
        # Backward propagation.
        #(≈ 1 line of code)
        # grads = ...    
        # YOUR CODE STARTS HERE
        grads = L_model_backward(AL, Y, caches)
        
        # YOUR CODE ENDS HERE
 
        # Update parameters.
        #(≈ 1 line of code)
        # parameters = ...
        # YOUR CODE STARTS HERE
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # YOUR CODE ENDS HERE
                
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [34]:
parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 500, print_cost = True)

Cost after iteration 0: 0.6931477726958
Cost after iteration 100: 0.6780107728090445
Cost after iteration 200: 0.6675997530076282
Cost after iteration 300: 0.6604218729413204
Cost after iteration 400: 0.6554578158288098
Cost after iteration 499: 0.6520420780037617


In [48]:
pred_train = predict(train_x, train_y, parameters)

Accuracy: 0.6555023923444976


In [47]:
pred_test = predict(test_x, test_y, parameters)

Accuracy: 0.34
